In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [2]:
path = 'C:\\Users\\Dan Siegel\\Desktop\\Classes\\550\\data\\wikipedia\\featured-articles\\featured-articles_000.jsonl'
with open(path) as f:
    lines = f.readlines()

articles = [json.loads(line) for line in lines]

In [3]:
corpus = articles[0:5]

In [4]:
df = pd.DataFrame.from_records(corpus)

In [155]:
def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    nopunc = nopunc.strip('\"')
    nopunc = nopunc.strip(',')
    nopunc = nopunc.strip('.')
    no_stopwords = [word for word in nopunc.split() if word not in stopwords.words('english')]
    nostops =  ' '.join(no_stopwords)
    return nostops.replace("\'", "")

In [95]:
text_process("\n\n\n \n\n\n\n\n'''Autism''' ....\"\"is a developmental disorder characterized by tro")

'autism developmental disorder characterized tro'

In [156]:
df['cleaned_text'] = df['section_texts'].apply(text_process)

In [161]:
df['cleaned_text'] = df['cleaned_text'].apply(text_process)

In [54]:
nltk.word_tokenize(df['cleaned_text'].iloc[0])

["'",
 "''",
 'autism',
 "''",
 "'",
 'developmental',
 'disorder',
 'characterized',
 'troubles',
 'social',
 'interaction',
 'communication',
 'restricted',
 'repetitive',
 'behavior',
 '.',
 'parents',
 'usually',
 'notice',
 'signs',
 'first',
 'two',
 'three',
 'years',
 'child',
 "'s",
 'life',
 '.',
 'signs',
 'often',
 'develop',
 'gradually',
 ',',
 'though',
 'children',
 'autism',
 'reach',
 'developmental',
 'milestones',
 'normal',
 'pace',
 'worsen',
 '.',
 'autism',
 'caused',
 'combination',
 'genetic',
 'environmental',
 'factors',
 '.',
 'risk',
 'factors',
 'include',
 'certain',
 'infections',
 'pregnancy',
 ',',
 'rubella',
 ',',
 'well',
 'valproic',
 'acid',
 ',',
 'alcohol',
 'cocaine',
 'use',
 'pregnancy',
 '.',
 'controversies',
 'surround',
 'proposed',
 'environmental',
 'causes',
 ',',
 'example',
 'vaccine',
 'hypotheses',
 ',',
 'disproven',
 '.',
 'autism',
 'affects',
 'information',
 'processing',
 'brain',
 'altering',
 'nerve',
 'cells',
 'synapses'

In [6]:
df['Tokenized'] = np.nan

In [168]:
for row in range(len(df)):
    df['Tokenized'].iloc[row] = nltk.word_tokenize(df['cleaned_text'].iloc[row])

In [10]:
def jaccard_distance(d1_words, d2_words):
    d1_unique = set(d1_words)
    d2_unique = set(d2_words)
    num_both = len(d1_unique.intersection(d2_unique))
    num_total = len(d1_unique.union(d2_unique))
    return num_both/num_total

In [11]:
jaccard_finalsims = []
for row in range(len(df)):
    row_1 = jaccard_distance(df['Tokenized'].iloc[row],df['Tokenized'].iloc[0])
    row_2 = jaccard_distance(df['Tokenized'].iloc[row],df['Tokenized'].iloc[1])
    row_3 = jaccard_distance(df['Tokenized'].iloc[row],df['Tokenized'].iloc[2])
    row_4 = jaccard_distance(df['Tokenized'].iloc[row],df['Tokenized'].iloc[3])
    row_5 = jaccard_distance(df['Tokenized'].iloc[row],df['Tokenized'].iloc[4])
    results = {'row': row, 'article 1':row_1, 'article 2': row_2, 'article 3': row_3, 'article 4': row_4, 'article 5': row_5}
    jaccard_finalsims.append(results)
jaccard_df = pd.DataFrame.from_records(jaccard_finalsims)

In [12]:
jaccard_df

,article 1,article 2,article 3,article 4,article 5,row
0,1.000000,0.135071,0.112044,0.119168,0.081127,0
1,0.135071,1.000000,0.115337,0.106605,0.087224,1
2,0.112044,0.115337,1.000000,0.104793,0.106511,2
3,0.119168,0.106605,0.104793,1.000000,0.083742,3
4,0.081127,0.087224,0.106511,0.083742,1.000000,4


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [73]:
df['TFIDF'] = np.nan

In [101]:
tf_vecd = TfidfVectorizer()

In [97]:
#df['TFIDF']  = tf_vecd.fit_transform(df['cleaned_text'])

In [93]:
jaccard_distance(df['TFIDF'].iloc[0],df['TFIDF'].iloc[0])

TypeError: unhashable type: 'csr_matrix'

In [111]:
tf_idfed_data = tf_vecd.fit_transform(df['cleaned_text'])

In [113]:
tf_idfed_data.getrow(0), tf_idfed_data.getrow(1), tf_idfed_data.getrow(2), tf_idfed_data.getrow(3), tf_idfed_data.getrow(4), 

(<1x6311 sparse matrix of type '<class 'numpy.float64'>'
 	with 1956 stored elements in Compressed Sparse Row format>,
 <1x6311 sparse matrix of type '<class 'numpy.float64'>'
 	with 2679 stored elements in Compressed Sparse Row format>,
 <1x6311 sparse matrix of type '<class 'numpy.float64'>'
 	with 1700 stored elements in Compressed Sparse Row format>,
 <1x6311 sparse matrix of type '<class 'numpy.float64'>'
 	with 1247 stored elements in Compressed Sparse Row format>,
 <1x6311 sparse matrix of type '<class 'numpy.float64'>'
 	with 1050 stored elements in Compressed Sparse Row format>)

In [19]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
import gensim, re
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

In [98]:
lda_vectors = CountVectorizer(lowercase=False)
lda_vects = lda_vectors.fit_transform(df.cleaned_text)

In [110]:
lda_model = LatentDirichletAllocation(n_topics=10)

In [111]:
lda_output = lda_model.fit_transform(lda_vects)

C:\Anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


'autism developmental disorder characterized troubles social interaction communication restricted repetitive behavior parents usually notice signs first two three years childs life signs often develop gradually though children autism reach developmental milestones normal pace worsen autism caused combination genetic environmental factors risk factors include certain infections pregnancy rubella well valproic acid alcohol cocaine use pregnancy controversies surround proposed environmental causes example vaccine hypotheses disproven autism affects information processing brain altering nerve cells synapses connect organize occurs well understood dsm5 autism included within autism spectrum asds along asperger syndrome less severe pervasive developmental disorder otherwise specified pddnos early speech behavioral interventions help children autism gain selfcare social skills communication skills although known cure cases children recovered condition many children autism live independently r

In [181]:
from gensim.models import LdaModel

In [126]:
from gensim import corpora, models

In [169]:
dictionary = corpora.Dictionary(df.Tokenized)

In [170]:
corpus = [dictionary.doc2bow(text) for text in df.Tokenized]

In [171]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)

In [214]:
topics = ldamodel.print_topics(num_topics=10, num_words=1)

In [235]:
lda_predictions = []
for i in range(len(df.Tokenized)):
    line_extract = df.Tokenized.iloc[i]
    new_review_bow = dictionary.doc2bow(line_extract)
    new_review_lda = ldamodel[new_review_bow]
    topic_no = new_review_lda[0][0]
    topic = re.findall('[A-z]\w+', topics[topic_no][1])[0]
    data = {'Article': i+1, 'Topic':topic, 'Method':'LdaModel'}
    lda_predictions.append(data)

# 2 B Latent Dirichlet Allocation

Create a topic model using Latent Dirichlet Allocation and report the results.



In [236]:
lda_predictions

[{'Article': 1, 'Topic': 'autism', 'Method': 'LdaModel'},
 {'Article': 2, 'Topic': 'amphibians', 'Method': 'LdaModel'},
 {'Article': 3, 'Topic': 'apollo', 'Method': 'LdaModel'},
 {'Article': 4, 'Topic': 'aikido', 'Method': 'LdaModel'},
 {'Article': 5, 'Topic': 'aries', 'Method': 'LdaModel'}]

In [237]:
from gensim.sklearn_api import lsimodel

In [285]:
latem_sem_cv = CountVectorizer()
lcv_words = latem_sem_cv.fit_transform(df.cleaned_text)
lcv_corp = gensim.matutils.Sparse2Corpus(lcv_words)

In [286]:
vocabulary_gensim = {}
for key, val in latem_sem_cv.vocabulary_.items():
    vocabulary_gensim[val] = key

In [291]:
latent_sem_mod = gensim.models.LsiModel(lcv_corp, num_topics=10, id2word=vocabulary_gensim)

# 2 A Latent Semantic Analysis

Create a topic model using Latent Semantic Analysis and report the results.

In [331]:
latent_sem_mod.print_topics(2)

[(0,
  '0.728*"00" + 0.615*"002" + 0.280*"01" + 0.083*"02" + 0.079*"03rd" + 0.000*"picked" + 0.000*"pictures" + 0.000*"physiological" + 0.000*"pi" + 0.000*"piano"'),
 (1,
  '-0.683*"00" + 0.647*"002" + 0.337*"01" + 0.038*"03rd" + 0.024*"02" + 0.000*"picked" + 0.000*"pictures" + 0.000*"physiological" + 0.000*"pi" + 0.000*"piano"')]

# 2 C Non-Negative Matrix Factorization

Create a topic model using Non-Negative Matrix Factorization and report the results

In [293]:
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;

In [300]:
NNMF_vectorizer = CountVectorizer()
NNMF_fitted_CV = NNMF_vectorizer.fit_transform(df.cleaned_text)

In [305]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(NNMF_fitted_CV);
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [315]:
NNMF_mod = NMF(n_components=5, init='nndsvd');
#fit the model
NNMF = NNMF_mod.fit(xtfidf_norm)

In [344]:
NNMF_ressults = []

In [349]:
for topic_idx, topic in enumerate(NNMF.components_):
        topic = " ".join([vocabulary_gensim[i]
                             for i in topic.argsort()[:-1 - 1:-1]])
        data = {'Article': topic_idx +1, 'Topic':topic, 'Method':'NNMF'}
        NNMF_ressults.append(data)

In [350]:
NNMF_ressults

[{'Article': 1, 'Topic': 'aries', 'Method': 'NNMF'},
 {'Article': 2, 'Topic': 'autism', 'Method': 'NNMF'},
 {'Article': 3, 'Topic': 'apollo', 'Method': 'NNMF'},
 {'Article': 4, 'Topic': 'aikido', 'Method': 'NNMF'},
 {'Article': 5, 'Topic': 'amphibians', 'Method': 'NNMF'}]

In [351]:
merged = NNMF_ressults + lda_predictions

In [352]:
merged

[{'Article': 1, 'Topic': 'aries', 'Method': 'NNMF'},
 {'Article': 2, 'Topic': 'autism', 'Method': 'NNMF'},
 {'Article': 3, 'Topic': 'apollo', 'Method': 'NNMF'},
 {'Article': 4, 'Topic': 'aikido', 'Method': 'NNMF'},
 {'Article': 5, 'Topic': 'amphibians', 'Method': 'NNMF'},
 {'Article': 1, 'Topic': 'autism', 'Method': 'LdaModel'},
 {'Article': 2, 'Topic': 'amphibians', 'Method': 'LdaModel'},
 {'Article': 3, 'Topic': 'apollo', 'Method': 'LdaModel'},
 {'Article': 4, 'Topic': 'aikido', 'Method': 'LdaModel'},
 {'Article': 5, 'Topic': 'aries', 'Method': 'LdaModel'}]